In [107]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn import preprocessing
import wfdb
import copy as cp
import scipy.signal as signal
import scipy.stats as stats
from sklearn import preprocessing
from tqdm import tqdm
import os
import re
import pandas as pd
import pickle
import csv
import statistics

In [2]:
rlist = []
records = os.path.normpath('mit-bih-dataframes/subject_list.csv')
with open(records) as rfile:
    recordreader = csv.reader(rfile, delimiter=' ', quotechar='|')
    for row in recordreader:
        rlist.append(row[0])

In [96]:
full_dfs = {}
for record in tqdm(rlist):
    full_dfs[record] = pd.read_csv(os.path.normpath('mit-bih-dataframes/'+record+'.csv'), index_col=[0])

100%|███████████████████████████████████████████| 23/23 [01:24<00:00,  3.66s/it]


In [3]:
subject_dfs = {}
for record in tqdm(rlist):
    subject_dfs[record] = pd.read_csv(os.path.normpath('mit-bih-extracted/'+record+'_full.csv'), index_col=[0,1])

100%|███████████████████████████████████████████| 23/23 [00:01<00:00, 12.35it/s]


In [4]:
subject_dfs['04015'].iloc[:100].head()

,,rr_int,rr_int_seconds,rhythmLabel,rmean,rmean_seconds,drmean,rr_variance
,timestamp,,,,,,,
0,0.000,139,0.556,N,139.000000,0.556000,0.000000,0.000000
1,0.632,158,0.632,N,143.750000,0.575000,4.750000,0.132174
2,1.536,226,0.904,N,164.312500,0.657250,20.562500,0.413846
3,2.116,145,0.580,N,159.484375,0.637938,-4.828125,0.507887
4,2.692,144,0.576,N,155.613281,0.622453,-3.871094,0.006426


In [179]:
def classify_rr_ints(df):
    #list of types of rr_ints for each subject
    subject_types = []
    for row in df.itertuples():
        if row.rrInt < 0.85*row.rmean: 
            #if rr_ints is less than 85% of runningmean
            #label subject type as short
            subject_types.append('short')
        elif row.rrInt > 1.15*row.rmean: 
            #if rr_ints is greater than 115% of runningmean
            #label subject type as long
            subject_types.append('long')
        else:
            #label subject type as regular
            subject_types.append('regular')
    
    return subject_types

In [6]:
interval_types = classify_rr_ints(subject_dfs['04015'].iloc[:100])
print(interval_types)

['regular', 'regular', 'long', 'regular', 'regular', 'long', 'regular', 'regular', 'long', 'regular', 'regular', 'regular', 'regular', 'regular', 'short', 'short', 'short', 'short', 'short', 'long', 'long', 'short', 'regular', 'long', 'short', 'regular', 'long', 'long', 'regular', 'regular', 'regular', 'regular', 'short', 'short', 'short', 'short', 'short', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'long', 'long', 'regular', 'short', 'long', 'short', 'long', 'regular', 'regular', 'long', 'regular', 'regular', 'long', 'short', 'regular', 'long', 'long', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'regular', 'short', 'short', 'short', 'short', 'regular', 'regular', 'regular', 'long', 'regular', 'regular', 'long', 'short', 'regular', 'long', 'short', 'short', 'long', 'short', 'short', 'short', 'long', 'long', 'short']


In [228]:
def find_proportions(int_types):
    StoS = 0
    StoR = 0
    StoL = 0
    RtoS = 0
    RtoR = 0
    RtoL = 0
    LtoS = 0
    LtoR = 0
    LtoL = 0
    for idx in range(len(int_types)):
        if idx<len(int_types)-1:
            if int_types[idx]=='short' and int_types[idx+1]=='short':
                StoS+=1
            elif int_types[idx]=='short' and int_types[idx+1]=='regular':
                StoR+=1
            elif int_types[idx]=='short' and int_types[idx+1]=='long':
                StoL+=1
            elif int_types[idx]=='regular' and int_types[idx+1]=='short':
                RtoS+=1
            elif int_types[idx]=='regular' and int_types[idx+1]=='regular':
                RtoR+=1
            elif int_types[idx]=='regular' and int_types[idx+1]=='long':
                RtoL+=1
            elif int_types[idx]=='long' and int_types[idx+1]=='short':
                LtoS+=1
            elif int_types[idx]=='long' and int_types[idx+1]=='regular':
                LtoR+=1
            elif int_types[idx]=='long' and int_types[idx+1]=='long':
                LtoL+=1
    
    count = len(int_types)-1
    subject_transitions = [StoS/count, StoR/count, StoL/count, RtoS/count, RtoR/count, RtoL/count, LtoS/count, LtoR/count, LtoL/count]
    
    return subject_transitions

In [67]:
rpeak_dfs = {}
for record in tqdm(rlist):
    rpeak_dfs[record] = pd.read_csv(os.path.normpath('mit-bih-extracted/'+record+'_rpeaks.csv'), names=['rpeak'])

100%|███████████████████████████████████████████| 23/23 [00:00<00:00, 63.56it/s]


In [68]:
rpeak_dfs['04015']

,rpeak
0,61.0
1,200.0
2,358.0
3,584.0
4,729.0
...,...
44000,8999121.0
44001,8999327.0
44002,8999531.0
44003,8999734.0


In [188]:
def extract_rmean(rrInts):
    rmeans = []
    for index, value in enumerate(rrInts):
        if index==0:
            rmeans.append(value)
        else:
            rmeans.append(0.75*rmeans[index-1] + 0.25*value)
    
    return rmeans

In [220]:
def extract_rmssd(subset):
    rrInts = subset['rrInt'].tolist()
    sum_of_squares = 0
    for idx, rrInt in enumerate(rrInts):
        if idx<len(rrInts)-1:
            square_difference = (rrInt-rrInts[idx-1])**2
            sum_of_squares+=square_difference
    mean_sum = sum_of_squares/(len(rrInts)-1)
    return np.sqrt(mean_sum)

In [262]:
def subset_subject(full_df, rpeak_df, interval_length = 10):
    rpeaks = rpeak_df['rpeak'].tolist()
    cutoff = interval_length*250

    current_subset = []
    subsets = []
    beginning_idx = 0
    outlier_comp = 0
    prev_peak = rpeaks[beginning_idx]
    for idx, peak in enumerate(rpeaks):
        if idx<len(rpeaks)-1:
            if peak<=cutoff+rpeaks[beginning_idx]+outlier_comp:
                if peak-prev_peak>500:
                    outlier_comp+=(peak-prev_peak)
                    prev_peak = peak
                else:
                    current_subset.append(peak)
                    prev_peak = peak
            else:
                rr_int_column = [current_subset[x]-current_subset[x-1] for x in range(1, len(current_subset))]
                rhythm_column = []
                for x in range(1, len(current_subset)):
                    if full_df['Normal'][current_subset[x]]:
                        rhythm_column.append('N')
                    elif full_df['AFIB'][current_subset[x]]:
                        rhythm_column.append('A')
                    elif full_df['Other'][current_subset[x]]:
                        rhythm_column.append('O')
                
                rmean_column = extract_rmean(rr_int_column)

                subsets.append(pd.DataFrame({'rhythmLabel': rhythm_column, 'rrInt': rr_int_column, 'rmean': rmean_column}, columns=['rhythmLabel', 'rrInt', 'rmean']))
                beginning_idx = idx
                current_subset = []
                outlier_comp = 0
                prev_peak = rpeaks[idx+1]

    return subsets

In [263]:
subset_dfs = {}
for record in tqdm(rlist):
    subset_dfs[record] = subset_subject(full_dfs[record], rpeak_dfs[record])

100%|███████████████████████████████████████████| 23/23 [01:18<00:00,  3.41s/it]


In [264]:
for record in tqdm(rlist):
    subsets = subset_dfs[record]
    idx_list = list(range(len(subsets)))
    
    data = {
        "subjectID": [record]*len(subsets),
        "subsetID": idx_list,
        "rhythmLabel": [subsets[x]['rhythmLabel'].mode()[0] for x in idx_list]
    }
    
    subset_list = pd.DataFrame(data)
    subset_list.to_csv(os.path.normpath('mit-bih-subsets/'+record+"_subset_list.csv"))
    
    os.makedirs('mit-bih-subsets/'+str(record), exist_ok=True)

    for x, subset in enumerate(subsets):
        subset.to_csv(os.path.normpath('mit-bih-subsets/'+str(record)+'/'+str(record)+"-"+str(idx_list[x])+".csv"))

100%|███████████████████████████████████████████| 23/23 [01:34<00:00,  4.10s/it]


In [8]:
def find_rrv(df):
    rrvs = [0]
    prev_rr_int = 0
    for idx, row in enumerate(df.itertuples()):
        if idx==0:
            prev_rr_int = row.rr_int
        else:
            rrvs.append(np.abs(row.rr_int-prev_rr_int)/row.rmean)
    return rrvs

In [224]:
def subset_features(subset_list):
    subset_dfs = {}
    for x, subset in enumerate(subset_list.itertuples()):
        subset_dfs[x] = pd.read_csv(os.path.normpath('mit-bih-subsets/'+str(subset.subjectID)+'/'+str(subset.subjectID)+"-"+str(x)+".csv"), index_col=0)

    feature_df = pd.DataFrame()

    props = [find_proportions(classify_rr_ints(subset_dfs[key])) for key in subset_dfs]
    feature_df['StoS'] = [x[0] for x in props]
    feature_df['StoR'] = [x[1] for x in props]
    feature_df['StoL'] = [x[2] for x in props]
    feature_df['RtoS'] = [x[3] for x in props]
    feature_df['RtoR'] = [x[4] for x in props]
    feature_df['RtoL'] = [x[5] for x in props]
    feature_df['LtoS'] = [x[6] for x in props]
    feature_df['LtoR'] = [x[7] for x in props]
    feature_df['LtoL'] = [x[8] for x in props]

    feature_df['std'] = [np.std(subset_dfs[key]['rrInt']) for key in subset_dfs]
    feature_df['cov'] = feature_df['std']/[np.mean(subset_dfs[key]['rrInt']) for key in subset_dfs]
    feature_df['range'] = [np.max(subset_dfs[key]['rrInt'])-np.min(subset_dfs[key]['rrInt']) for key in subset_dfs]
    #feature_df['rmean'] = df['rmean'].tolist()
    #feature_df['rrv'] = df['rr_variance'].tolist()
    feature_df['rrInt_var'] = [subset_dfs[key]['rrInt'].var() for key in subset_dfs]
    feature_df['rmean_var'] = [subset_dfs[key]['rmean'].var() for key in subset_dfs]
    feature_df['rmssd'] = [extract_rmssd(subset_dfs[key]) for key in subset_dfs]
    feature_df['mad'] = [stats.median_abs_deviation(subset_dfs[key]['rrInt']) for key in subset_dfs]
    feature_df['iqr'] = [stats.iqr(subset_dfs[key]['rrInt']) for key in subset_dfs]

    return pd.concat([subset_list, feature_df], axis=1)

In [265]:
features_df = {}
for record in tqdm(rlist):
    subset_list = pd.read_csv('mit-bih-subsets/'+record+'_subset_list.csv', index_col=0, dtype={'subjectID': str})
    features = subset_features(subset_list)

    features.to_csv(os.path.normpath('mit-bih-features/'+record+".csv"))

100%|███████████████████████████████████████████| 23/23 [03:27<00:00,  9.01s/it]
